In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
import os


# Perfect DHard for UPMC

In [2]:
# Per sample predictions for new DHard

In [3]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_food101_full_101_f739c19e1aeeaea18c60b1bf802b05db"

runs = [
        "MT_Augmented_49ac1ef407c36a9902d074d199ed6af2/Metrics"
]

stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/common_data/food_101/DF/df_food101_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['French onion soup','Hot and sour soup','Pho','Takoyaki', 'Churros','Beignets', 'Lobster bisque', 'Sashimi', 'Clam chowder','Onion rings']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [84.388120028052]
Mean accuracy across runs :  84.388120028052
Mean Acc for thrashed classes : 79.96000213623047


In [4]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [5]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 58.70091285705566
Mean dhard_sub performance 100.0
Mean acc_improvement 79.96000213623047


In [6]:
stats['accuracy']['dhard_best'][41]

64.75409698486328

In [7]:
df_dhard = pd.read_pickle("/bigstor/zsarwar/Enola_Augmented/MT_food101_full_101_f739c19e1aeeaea18c60b1bf802b05db/Datasets/df_food101_val_test_empirical_ac94cbbad3e3548851c66a600665e5e8.pkl")
#df_dhard_sub = pd.read_pickle("/bigstor/zsarwar/Enola_Augmented/MT_food101_full_101_f739c19e1aeeaea18c60b1bf802b05db/Datasets/df_food101_val_test_sub_empirical_ac94cbbad3e3548851c66a600665e5e8.pkl")

In [8]:
labels = df_dhard['label'].unique().tolist()

In [9]:
df_dhard_sub = None
for curr_label in labels:
    df_dhard_temp = df_dhard[df_dhard['label'] == curr_label]
    label_preds = stats['predicted_labels']['dhard_best'][curr_label]
    label_preds = np.asarray(label_preds)
    correct_indices = np.where(label_preds == curr_label)[0]
    # Sub-select dhard
    df_dhard_sub_temp = df_dhard_temp.iloc[correct_indices]
    if isinstance(df_dhard_sub, pd.DataFrame):
        frames = [df_dhard_sub, df_dhard_sub_temp]
        df_dhard_sub = pd.concat(frames)
    else:
        df_dhard_sub = df_dhard_sub_temp
        

In [10]:
# Per class budget
resample = [86, 97, 23]
per_class_budget = 32
seed = 42
seed2 = 11
df_dhard_subsampled = None
for curr_label in labels:
    df_temp = df_dhard_sub[df_dhard_sub['label'] == curr_label]
    if curr_label in resample:
        df_sub = df_temp.sample(n=per_class_budget, random_state=seed2)
    else:
        df_sub = df_temp.sample(n=per_class_budget, random_state=seed)
    if isinstance(df_dhard_subsampled, pd.DataFrame):
        frames = [df_dhard_subsampled, df_sub]
        df_dhard_subsampled = pd.concat(frames)
    else:
        df_dhard_subsampled = df_sub


In [11]:
df_dhard_subsampled.to_pickle("/bigstor/zsarwar/Enola_Augmented/MT_food101_full_101_f739c19e1aeeaea18c60b1bf802b05db/Datasets/df_food101_val_test_sub_empirical_ac94cbbad3e3548851c66a600665e5e8.pkl")

In [12]:
pd.read_pickle("/bigstor/zsarwar/Enola_Augmented/MT_food101_full_101_f739c19e1aeeaea18c60b1bf802b05db/Datasets/df_food101_val_test_sub_empirical_ac94cbbad3e3548851c66a600665e5e8.pkl")

,image_url,class,label,pred_label,data_type,dataset,query,img_path
french_onion_soup/1465084.jpg,None,French onion soup,41,22,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...
french_onion_soup/1184744.jpg,None,French onion soup,41,81,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...
french_onion_soup/2709614.jpg,None,French onion soup,41,59,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...
french_onion_soup/723360.jpg,None,French onion soup,41,27,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...
french_onion_soup/2943991.jpg,None,French onion soup,41,64,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...
...,...,...,...,...,...,...,...,...
lobster_bisque/196433.jpg,None,Lobster bisque,60,73,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...
lobster_bisque/2770542.jpg,None,Lobster bisque,60,16,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...
lobster_bisque/2390410.jpg,None,Lobster bisque,60,27,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...
lobster_bisque/2954812.jpg,None,Lobster bisque,60,87,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...


# Perfect DHard for Imagenet120

In [2]:
# Per sample predictions for new DHard

In [3]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_Imagenet_dogs_120_44494fed02dee22a95cced5d0322a3ed"


runs = [
        "MT_Augmented_17f4761e9fca96f98ddbbf212e18b2e8/Metrics"
]

stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_Imagenet_dogs_120_44494fed02dee22a95cced5d0322a3ed/Datasets/df_imagenet_dogs_val_test.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['Welsh springer spaniel','Bedlington terrier','Pomeranian','pug, pug-dog','African hunting dog, hyena dog, Cape hunting dog, Lycaon pictus','Bernese mountain dog','Boston bull, Boston terrier','Leonberg','Samoyed, Samoyede','borzoi, Russian wolfhound']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [78.91666666666667]
Mean accuracy across runs :  78.91666666666667
Mean Acc for thrashed classes : 79.8


# Analysis of DHard and DHard_sub

In [4]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [5]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
index,,,,,
ILSVRC2012_val_00000057.JPEG,Pomeranian,107,75.000000,75.0,92.0
ILSVRC2012_val_00000078.JPEG,"African hunting dog, hyena dog, Cape hunting d...",119,100.000000,100.0,100.0
ILSVRC2012_val_00000220.JPEG,"pug, pug-dog",102,80.000000,75.0,94.0
ILSVRC2012_val_00000350.JPEG,"Boston bull, Boston terrier",44,28.571428,50.0,62.0
ILSVRC2012_val_00000590.JPEG,"Samoyed, Samoyede",106,55.555553,50.0,90.0
ILSVRC2012_val_00000883.JPEG,Bedlington terrier,30,56.521736,50.0,80.0
ILSVRC2012_val_00001203.JPEG,Leonberg,103,85.714287,75.0,98.0
ILSVRC2012_val_00001626.JPEG,"borzoi, Russian wolfhound",18,57.575756,50.0,70.0
ILSVRC2012_val_00002298.JPEG,Welsh springer spaniel,67,50.000000,50.0,60.0


In [6]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 63.218200492858884
Mean dhard_sub performance 65.0
Mean acc_improvement 79.8


In [8]:
df_dhard = pd.read_pickle("/bigstor/zsarwar/Enola_Augmented/MT_Imagenet_dogs_120_44494fed02dee22a95cced5d0322a3ed/Datasets/df_imagenet_dogs_val_test_empirical_acd44d96378cab159d240eb4c7a9c98c.pkl")


In [9]:
labels = df_dhard['label'].unique().tolist()

In [10]:
df_dhard_sub = None
for curr_label in labels:
    df_dhard_temp = df_dhard[df_dhard['label'] == curr_label]
    label_preds = stats['predicted_labels']['dhard_best'][curr_label]
    label_preds = np.asarray(label_preds)
    correct_indices = np.where(label_preds == curr_label)[0]
    # Sub-select dhard
    df_dhard_sub_temp = df_dhard_temp.iloc[correct_indices]
    if isinstance(df_dhard_sub, pd.DataFrame):
        frames = [df_dhard_sub, df_dhard_sub_temp]
        df_dhard_sub = pd.concat(frames)
    else:
        df_dhard_sub = df_dhard_sub_temp
        

In [11]:
# Per class budget
#resample = [86, 97, 23]
per_class_budget = 4
seed = 42
seed2 = 11
df_dhard_subsampled = None
for curr_label in labels:
    df_temp = df_dhard_sub[df_dhard_sub['label'] == curr_label]
    #if curr_label in resample:
        #df_sub = df_temp.sample(n=per_class_budget, random_state=seed2)
    #else:
    df_sub = df_temp.sample(n=per_class_budget, random_state=seed)
    if isinstance(df_dhard_subsampled, pd.DataFrame):
        frames = [df_dhard_subsampled, df_sub]
        df_dhard_subsampled = pd.concat(frames)
    else:
        df_dhard_subsampled = df_sub


In [14]:
df_dhard_subsampled.to_pickle("/bigstor/zsarwar/Enola_Augmented/MT_Imagenet_dogs_120_44494fed02dee22a95cced5d0322a3ed/Datasets/df_imagenet_dogs_val_test_sub_empirical_acd44d96378cab159d240eb4c7a9c98c.pkl")

In [15]:
pd.read_pickle("/bigstor/zsarwar/Enola_Augmented/MT_Imagenet_dogs_120_44494fed02dee22a95cced5d0322a3ed/Datasets/df_imagenet_dogs_val_test_sub_empirical_acd44d96378cab159d240eb4c7a9c98c.pkl")

,image_url,class,label,pred_label,data_type,dataset,query,img_path
index,,,,,,,,
ILSVRC2012_val_00045514.JPEG,None,Pomeranian,107,0,Imagenet,Imagenet,None,/bigstor/zsarwar/Imagenet_2012/val/ILSVRC2012_...
ILSVRC2012_val_00003583.JPEG,None,Pomeranian,107,79,Imagenet,Imagenet,None,/bigstor/zsarwar/Imagenet_2012/val/ILSVRC2012_...
ILSVRC2012_val_00024277.JPEG,None,Pomeranian,107,0,Imagenet,Imagenet,None,/bigstor/zsarwar/Imagenet_2012/val/ILSVRC2012_...
ILSVRC2012_val_00000057.JPEG,None,Pomeranian,107,3,Imagenet,Imagenet,None,/bigstor/zsarwar/Imagenet_2012/val/ILSVRC2012_...
ILSVRC2012_val_00001637.JPEG,None,"African hunting dog, hyena dog, Cape hunting d...",119,118,Imagenet,Imagenet,None,/bigstor/zsarwar/Imagenet_2012/val/ILSVRC2012_...
ILSVRC2012_val_00004276.JPEG,None,"African hunting dog, hyena dog, Cape hunting d...",119,99,Imagenet,Imagenet,None,/bigstor/zsarwar/Imagenet_2012/val/ILSVRC2012_...
ILSVRC2012_val_00048385.JPEG,None,"African hunting dog, hyena dog, Cape hunting d...",119,23,Imagenet,Imagenet,None,/bigstor/zsarwar/Imagenet_2012/val/ILSVRC2012_...
ILSVRC2012_val_00007075.JPEG,None,"African hunting dog, hyena dog, Cape hunting d...",119,54,Imagenet,Imagenet,None,/bigstor/zsarwar/Imagenet_2012/val/ILSVRC2012_...
ILSVRC2012_val_00011495.JPEG,None,"pug, pug-dog",102,92,Imagenet,Imagenet,None,/bigstor/zsarwar/Imagenet_2012/val/ILSVRC2012_...


In [ ]:
resample = [86, 97, 23]